In [5]:
import pandas as pd
import math

In [6]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [12]:
sigmoid(2)

0.8807970779778823

In [13]:
def simple_perceptron(weights, bias):
    return  weights[0]*0.4 + weights[1]*0.6 +bias

In [21]:
weights=[3 ,5]
bias = -2.2

In [22]:
sigmoid(simple_perceptron(weights, bias))

0.8807970779778823